In [1]:
# --- Environment (set before importing jax) ---
import os
# Disable XLA preallocation (helps avoid "only 125MB free" situations)
os.environ.setdefault("XLA_PYTHON_CLIENT_PREALLOCATE", "false")
# Optional: if you want the CPU to be available alongside TPU
os.environ.setdefault("JAX_PLATFORMS", "cpu,tpu")

import gc
import warnings
warnings.filterwarnings("ignore", message=".*deprecated.*will be removed in Transformers v5.*")

import numpy as np
import jax
import jax.numpy as jnp
import torch
from transformers import AutoModelForCausalLM, AutoTokenizer, FlaxAutoModelForCausalLM, AutoConfig

print("JAX devices:", jax.devices())

model_id = "gemmathon/gemma-pro-3.1b-ko-v0.1"
local_dir = "./gemma_pt_singlefile"

# ---------------------------
# Step 1: Load PyTorch and save a single .bin (bfloat16) locally
# ---------------------------
def prepare_singlefile_pt_checkpoint(model_id, out_dir):
    os.makedirs(out_dir, exist_ok=True)

    # Skip if already exists
    bin_path = os.path.join(out_dir, "pytorch_model.bin")
    config_path = os.path.join(out_dir, "config.json")
    if os.path.exists(bin_path) and os.path.exists(config_path):
        print(f"Found existing single-file checkpoint at {out_dir}, skipping conversion.")
        return

    print("Loading PyTorch model (CPU) and saving single-file .bin locally...")
    tokenizer = AutoTokenizer.from_pretrained(model_id)

    # Load on CPU in bfloat16 to reduce RAM and file size
    pytorch_model = AutoModelForCausalLM.from_pretrained(
        model_id,
        torch_dtype=torch.bfloat16,
        low_cpu_mem_usage=True,
        device_map="cpu",
    )

    pytorch_model.save_pretrained(
        out_dir,
        safe_serialization=False,   # ensure .bin (not safetensors)
        max_shard_size="100GB",     # force single file
    )
    tokenizer.save_pretrained(out_dir)

    del pytorch_model
    gc.collect()
    if torch.cuda.is_available():
        torch.cuda.empty_cache()

prepare_singlefile_pt_checkpoint(model_id, local_dir)

# ---------------------------
# Step 2: Load Flax model from local .bin, conversion on CPU
# ---------------------------
def load_flax_model_from_local(local_dir):
    tokenizer = AutoTokenizer.from_pretrained(local_dir)
    if tokenizer.pad_token is None:
        tokenizer.pad_token = tokenizer.eos_token
    tokenizer.padding_side = "left"

    # Force conversion/allocation on CPU to avoid TPU OOM
    cpu = jax.devices("cpu")[0]
    print("Converting PT->Flax on CPU ...")
    with jax.default_device(cpu):
        model = FlaxAutoModelForCausalLM.from_pretrained(
            local_dir,
            from_pt=True,           # convert local pytorch_model.bin to Flax
            dtype=jnp.bfloat16,     # keep everything in bfloat16 for TPU
            local_files_only=True,
        )
    print("Flax model loaded (params on CPU).")
    return model, tokenizer

model, tokenizer = load_flax_model_from_local(local_dir)
params_cpu = model.params  # keep params on CPU; they will be transferred to TPU only when needed

# ---------------------------
# Step 3: Memory-aware generation utilities
# ---------------------------

def bytes_human(n):
    for unit in ["B","KB","MB","GB","TB"]:
        if n < 1024:
            return f"{n:.2f}{unit}"
        n /= 1024
    return f"{n:.2f}PB"

def tree_nbytes(tree):
    # Sum nbytes for all arrays in a pytree (CPU arrays here)
    total = 0
    for x in jax.tree_util.tree_leaves(tree):
        # jnp / numpy arrays
        if hasattr(x, "nbytes"):
            total += x.nbytes
        else:
            # possibly a Python scalar
            pass
    return total

def estimate_kv_cache_bytes(config, total_seq_len, batch_size=1, dtype_bytes=2):
    L = getattr(config, "num_hidden_layers", None)
    H = getattr(config, "num_attention_heads", None)
    KVH = getattr(config, "num_key_value_heads", None) or H
    d_model = getattr(config, "hidden_size", None)
    if None in (L, H, d_model):
        # Fallback rough estimate if config lacks fields
        # This is conservative but avoids surprises.
        return int(2 * 24 * 16 * (2048 // 16) * total_seq_len * batch_size * dtype_bytes)

    head_dim = d_model // H
    # 2 for K and V
    return int(batch_size * L * KVH * head_dim * total_seq_len * 2 * dtype_bytes)

def fit_seq_len_for_hbm(config, params_bytes, input_len, batch_size=1,
                        hbm_bytes=int(16 * 1024**3), safety_bytes=int(3 * 1024**3),
                        dtype_bytes=2, requested_new_tokens=128):
    """
    Returns a safe (possibly reduced) max_new_tokens to avoid OOM,
    using a conservative KV-cache estimate plus safety margin.
    """
    # Per-token KV memory (for one token)
    # We'll solve for total_seq_len: input_len + new_tokens
    def kv_bytes_for(total_len):
        return estimate_kv_cache_bytes(config, total_len, batch_size=batch_size, dtype_bytes=dtype_bytes)

    # Reserve space for params + safety (XLA scratch, temporary buffers, compilation)
    budget = hbm_bytes - params_bytes - safety_bytes
    if budget <= 0:
        # If this happens, reduce safety and retry
        budget = max(int(hbm_bytes * 0.3), int(hbm_bytes * 0.1))

    # Find maximum total_seq_len such that kv_bytes <= budget
    # Use a simple incremental expansion then binary search
    lo = input_len
    hi = max(input_len + requested_new_tokens, input_len + 1)
    while kv_bytes_for(hi) < budget:
        hi *= 2
        if hi > input_len + 4096:
            break

    # Binary search between lo..hi for max feasible
    while lo + 1 < hi:
        mid = (lo + hi) // 2
        if kv_bytes_for(mid) <= budget:
            lo = mid
        else:
            hi = mid

    allowed_total_len = lo
    allowed_new_tokens = max(0, allowed_total_len - input_len)
    # Never exceed user request
    allowed_new_tokens = min(allowed_new_tokens, requested_new_tokens)
    return allowed_new_tokens, allowed_total_len, budget

def summarize_memory_and_fit(model, params_cpu, tokenizer, prompt, requested_new_tokens=128, batch_size=1):
    config = AutoConfig.from_pretrained(local_dir)
    params_bytes = tree_nbytes(params_cpu)

    input_ids = tokenizer(prompt, return_tensors="np")["input_ids"]
    input_len = int(input_ids.shape[-1])

    allowed_new_tokens, allowed_total_len, budget = fit_seq_len_for_hbm(
        config=config,
        params_bytes=params_bytes,
        input_len=input_len,
        batch_size=batch_size,
        hbm_bytes=int(16 * 1024**3),   # v5e single core HBM
        safety_bytes=int(3 * 1024**3), # leave ~3GB for scratch/compile
        dtype_bytes=2,                 # bfloat16
        requested_new_tokens=requested_new_tokens,
    )

    print("---- Memory plan ----")
    print(f"Params (CPU): {bytes_human(params_bytes)}")
    per_token_kv = estimate_kv_cache_bytes(config, 1, batch_size=batch_size, dtype_bytes=2)
    print(f"KV cache per token: {bytes_human(per_token_kv)}")
    print(f"HBM budget for KV (after params+safety): {bytes_human(budget)}")
    print(f"Input length: {input_len} tokens")
    print(f"Requested new tokens: {requested_new_tokens}")
    print(f"Allowed total length: {allowed_total_len} tokens")
    print(f"Allowed new tokens: {allowed_new_tokens}")
    return allowed_new_tokens

def safe_generate(model, tokenizer, params_cpu, prompt, requested_new_tokens=128):
    # Compute a safe cap for new tokens
    new_tok = summarize_memory_and_fit(model, params_cpu, tokenizer, prompt, requested_new_tokens)

    inputs = tokenizer(prompt, return_tensors="np")
    # We pass params explicitly (CPU), JAX will transfer to TPU as needed
    # Try greedy decoding. If OOM at runtime, backoff and retry.
    for trial in range(4):
        try:
            outputs = model.generate(
                **inputs,
                params=params_cpu,
                max_new_tokens=max(1, new_tok),
                do_sample=False,        # greedy for minimal memory
                num_beams=1,
                pad_token_id=tokenizer.pad_token_id,
                eos_token_id=tokenizer.eos_token_id,
                return_dict_in_generate=True,
            )
            return tokenizer.decode(outputs.sequences[0], skip_special_tokens=True)
        except Exception as e:
            msg = str(e)
            if "RESOURCE_EXHAUSTED" in msg or "OutOfMemory" in msg or "RESOURCE_EXHAUSTED" in repr(e):
                # Back off by 35% and retry
                print(f"OOM at {new_tok} new tokens. Backing off...")
                new_tok = max(1, int(new_tok * 0.65))
                gc.collect()
            else:
                raise

    raise RuntimeError("Failed to generate even after retries.")

# ---------------------------
# Step 4: Test generation with memory-aware settings
# ---------------------------
prompt = "안녕"
text = safe_generate(model, tokenizer, params_cpu, prompt, requested_new_tokens=256)
print(text)

/usr/local/lib/python3.12/dist-packages/jax/_src/cloud_tpu_init.py:82: UserWarning: Transparent hugepages are not enabled. TPU runtime startup and shutdown time should be significantly improved on TPU v5e and newer. If not already set, you may need to enable transparent hugepages in your VM image (sudo sh -c "echo always > /sys/kernel/mm/transparent_hugepage/enabled")
  warnings.warn(
/usr/local/lib/python3.12/dist-packages/torch_xla/experimental/gru.py:113: SyntaxWarning: invalid escape sequence '\_'
  * **h_n**: tensor of shape :math:`(D * \text{num\_layers}, H_{out})` or


JAX devices: [CpuDevice(id=0)]
Loading PyTorch model (CPU) and saving single-file .bin locally...


tokenizer_config.json: 0.00B [00:00, ?B/s]

tokenizer.model:   0%|          | 0.00/4.24M [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/17.5M [00:00<?, ?B/s]

...

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

generation_config.json:   0%|          | 0.00/132 [00:00<?, ?B/s]

Converting PT->Flax on CPU ...


TensorFlow and JAX classes are deprecated and will be removed in Transformers v5. We recommend migrating to PyTorch classes or pinning your version of Transformers.
Gemma's activation function should be approximate GeLU and not exact GeLU. Changing the activation function to `gelu_pytorch_tanh`.if you want to use the legacy `gelu`, edit the `model.config` to set `hidden_activation=gelu`   instead of `hidden_act`. See https://github.com/huggingface/transformers/pull/29402 for more details.
Some weights of the model checkpoint at ./gemma_pt_singlefile were not used when initializing FlaxGemmaForCausalLM: {('lm_head', 'kernel')}
- This IS expected if you are initializing FlaxGemmaForCausalLM from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing FlaxGemmaForCausalLM from the checkpoint of a model that you expect to be exactl

Flax model loaded (params on CPU).
---- Memory plan ----
Params (CPU): 5.90GB
KV cache per token: 24.00KB
HBM budget for KV (after params+safety): 7.10GB
Input length: 3 tokens
Requested new tokens: 256
Allowed total length: 4143 tokens
Allowed new tokens: 256
안녕하세요.
이번에 2019년 1월 1일부터 2020년 1월 1일까지 1년간 1주일마다 1주일의 주제를 정하고 1주일마다 1주일의 주제를 정하고 1주일마다 1주일의 주제를 정하고 1주일마다 1주일의 주제를 정하고 1주일마다 1주일의 주제를 정하고 1주일마다 1주일의 주제를 정하고 1주일마다 1주일의 주제를 정하고 1주일마다 1주일의 주제를 정하고 1주일마다 1주일의 주제를 정하고 1주일마다 1주일의 주제를 정하고 1주일마다 1주일의 주제를 정하고 1주일마다 1주일의 주제를 정하고 1주일마다 1주일의 주제를 정하고 1주일마다 1주일의 주
